In [1]:
! pip install pinecone-client==2.2.4 

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [8]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import ConversationChain
from langchain_community.vectorstores import Qdrant
from langchain.memory import ConversationSummaryMemory
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.chat_models import ChatMlflow

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain import PromptTemplate

from domino_data.vectordb import DominoPineconeConfiguration

import os
import pinecone

In [9]:
# Load the embedding model
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/code/model_cache/'
embeddings = HuggingFaceBgeEmbeddings(model_name=embedding_model_name,
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

In [10]:
# Initialize Pinecone index
datasource_name = "Rakuten"
conf = DominoPineconeConfiguration(datasource=datasource_name)
api_key = os.environ.get("DOMINO_VECTOR_DB_METADATA", datasource_name)

pinecone.init(
    api_key=api_key,
    environment="domino",
    openapi_config=conf
)

index = pinecone.Index("rakuten")


In [11]:
chat = ChatMlflow(
    target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
    endpoint="chat-gpt35turbo-sm",
)

conversation_openai = ConversationChain(
    llm=chat,
    memory=ConversationSummaryMemory(llm=chat),
    verbose=False
)

conversation_anthropic = ConversationChain(
        llm=ChatAnthropic(model='claude-2.1'),
        memory=ConversationSummaryMemory(llm=ChatAnthropic(model='claude-2.1')),
        verbose=False
    )

messages = [
    {"role": "assistant", "content": "How can I help you today?"}
]

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [16]:
# Get relevant docs through vector DB

SIMILARITY_THRESHOLD = 0.83

# Number of texts to match (may be less if no suitable match)
NUM_TEXT_MATCHES = 3

# Create prompt
template = """ You are a virtual assistant for Rakuten and your task is to answer questions related to Rakuten which includes general information about Rakuten.

                Respond in the style of a polite helpful assistant and do not allude that you have looked up the context.

                Do not hallucinate. If you don't find an answer, you can point user to the official website here: https://www.rakuten.com/help . 

                In your response, include the following url links at the end of your response {url_links} and any other relevant URL links that you refered.

                Also, at the end of your response, ask if your response was helpful". 

                Here is some relevant context: {context}"""


# Get relevant docs through vector DB
def get_relevant_docs(user_input):
   
    embedded_query = embeddings.embed_query(user_input)
    
    relevant_docs = index.query(
        vector=embedded_query,
        top_k=NUM_TEXT_MATCHES,
        include_values=True,
        include_metadata=True
    )

    matches = relevant_docs["matches"]
    filtered_matches = [match for match in matches if match['score'] >= SIMILARITY_THRESHOLD]
    relevant_docs["matches"] = filtered_matches

    return relevant_docs

 
def build_system_prompt(user_input):
    
    relevant_docs = get_relevant_docs(user_input)
    
    actual_num_matches = len(relevant_docs["matches"])
    urls = set([relevant_docs["matches"][i]["metadata"]["source"] for i in range(actual_num_matches)])
    contexts = [relevant_docs["matches"][i]["metadata"]["text"] for i in range(actual_num_matches)]
    
    # TODO : pull the prompt template from the Hub
    
    # Create prompt
    template = """ You are a virtual assistant for Rakuten and your task is to answer questions related to Rakuten which includes general information about Rakuten.

                Respond in the style of a polite helpful assistant and do not allude that you have looked up the context.

                Do not hallucinate. If you don't find an answer, you can point user to the official website here: https://www.rakuten.com/help . 

                In your response, include the following url links at the end of your response {url_links} and any other relevant URL links that you refered.

                Also, at the end of your response, ask if your response was helpful". 

                Here is some relevant context: {context}"""
 
    prompt_template = PromptTemplate(
        input_variables=["url_links", "context"],
        template=template
    )
    
    system_prompt = prompt_template.format( url_links=urls, context=contexts)
 
    return system_prompt
 
# Query the Open AI Model
def queryAIModel(user_input, llm_name="openai"):

    system_prompt = build_system_prompt(user_input)            
    messages = [
        SystemMessage(
            content=system_prompt
        ),
        HumanMessage(
            content=user_input
        ),
    ]
    if llm_name.lower() == "openai":
        output = conversation_openai.predict(input=messages)
    else:
        output = conversation_anthropic.predict(input=messages)

    return output

In [18]:
# Ask a question ; uncomment to test
# user_question = input("Please provide your question here :")
# result = queryAIModel(user_question)
# result

Please provide your question here : where can i get a gift card?


'Hello! You can purchase gift cards from the Rakuten Gift Card Shop. The Rakuten Gift Card Shop offers a variety of third-party gift cards, gift codes, and other stored-value products. You can find more information about purchasing gift cards and the terms of sale at the official Rakuten help page here: [Rakuten Gift Card Shop Terms of Sale](https://www.rakuten.com/help/article/rakuten-gift-card-shop-terms-of-sale-360052462534). If you have any specific questions or need further assistance, feel free to reach out. Was this information helpful to you?'